# Rule Based System applied to Tarot

## Foreword
The idea here is to implement an ensemble of rules and conventions in French Tarot* so that the players make oriented decisions, and increase their chances of victory.

The main features of this 'Rule based system' can be found :
- in Player class :
    - open() -> "What should I do if I must play the first card of the trick ?"
    - answer() -> "What should I respond if cards have already been played ?"
- in Game class :
    - estimate_win_factor() -> "How great are my chances of victory ?"
    - incorporate_dog() -> "If I am Taker and may realize an 'ecart', what cards should be put aside first ?"
- in Tricks class :
    - compute_cut_risk() -> "Is playing this card safe ?"

*French Tarot rules are available here : https://www.pagat.com/tarot/frtarot.html

There are many cases and exceptions in every tarot game, my tests cover the great majority of them but some may have been left aside, so don't worry too much if you encounter a bug.

### Imports

In [1]:
import random
import pandas as pd
import statistics

## Game Modelisation

Are needed :
- trump cards + the fool + oudlers
- figures = King, Queen, Paladin, Jack
- non-figure cards = 1 to 10
- colors = hearts, clubs, diamonds and spades


Steps :
- Dealing
- Biding
- Handful
- Play of the card

## Cards & Bids Definition

In [2]:
### CARDS ###
FIGURES = ['J','P','Q','K']
FIGURES_NAMES = ["Jack","Knight","Queen","King"]
NON_FIGURES = ['1','2','3','4','5','6','7','8','9','10']
CARD_VALUES = NON_FIGURES + FIGURES

COLOR_VALUES = [ 'H', 'C', 'D', 'S']
COLOR_NAMES = ["Hearts","Clubs","Diamonds","Spades"]

TRUMPS = [ i for i in range(0,22)]
OUDLERS = [0, 1, 21]

COLOR_CARDS = [ COLOR_VALUES[j]+str(CARD_VALUES[i]) for j in range(4) for i in range(len(CARD_VALUES))]

PLAYING_CARDS = TRUMPS + COLOR_CARDS

### BIDS ###

BIDS=['PASS','TAKE','PUSH','GUARD','GUARD WITHOUT','GUARD AGAINST']    
BID_VALUES=[0,10,20,40,60,80]
# quantiles are computed from 1345 takings out of 10K bidings
QUANTILES=[1.005, 1.015, 1.035, 1.045, 1.065, 1.075, 1.105, 1.135, 1.185]
ROLES=['TAKER','OPENER','NEUTRAL','RELAUNCHER']

## Cards Recognition

In [3]:
### CARD FUNCTIONS ###
def get_card_role(card):
    # TODO : check else clause validity
    return card[1:] if card not in TRUMPS else card

def is_figure(card):
    return get_card_role(card) in FIGURES

def is_non_figure(card):
    return get_card_role(card) in NON_FIGURES

def is_trump(card):
    return type(card) is int

def is_oudler(card):
    return card in OUDLERS

def get_color(card):
    return card[0] if not is_trump(card) else None


### TESTS ###
sampling = random.sample(PLAYING_CARDS,2)

for card in sampling:
    print(f"\n=== Card : {card} ===\nCard value : {get_card_role(card)}\nis figure : {is_figure(card)}\nis not figure {is_non_figure(card)}\nis trump : {is_trump(card)}\nis oudler : {is_oudler(card)}\nhas color : {get_color(card)}")


=== Card : H8 ===
Card value : 8
is figure : False
is not figure True
is trump : False
is oudler : False
has color : H

=== Card : HQ ===
Card value : Q
is figure : True
is not figure False
is trump : False
is oudler : False
has color : H


## Hand Management : Sorting, Showing, Information

In [4]:
# UTILITY FUNCTION
def get_index(card):
    '''Utility function used to sort cards.'''
    return PLAYING_CARDS.index(card)

# UTILITY FUNCTION
def get_card_value(card):
    return (get_index(card)-22)%14+1 if not is_trump(card) else 14+card if card!=0 else 0

# UTILITY FUNCTION
def sort_cards(hand):
    return hand.sort(key=get_index)

# UI FUNCTION
def to_string(card):
    if is_trump(card):
        if card != 0:
            return(f"Trump {card}")
        else:
            return(f"Trump Fool")
    else:
        return(f"{COLOR_NAMES[COLOR_VALUES.index(card[0])]} {FIGURES_NAMES[FIGURES.index(card[1:])] if is_figure(card) else card[1:]}")

# UI FUNCTION
def show_cards(cards):
    for card in cards:
        print(to_string(card))

# HAND FUNCTION
def get_trump_cards(hand):
    return [card for card in hand if is_trump(card)]
def get_trump_count(hand):
    return sum([1 for card in get_trump_cards(hand)])

# HAND FUNCTION
def get_color_cards(hand,color):
    return [card for card in hand if get_color(card)==color]
def get_color_count(hand,color):
    return sum([1 for card in get_color_cards(hand,color)])

### TESTS ###
# TODO implement tests
for card in PLAYING_CARDS:
    print(get_card_value(card))

0
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
9
10
11
12
13
14


## Trick Management

In [5]:
def holder(trick):
    '''
    Is holding the highest card in opening color or biggest trump.
    Returns the card holding the trick so far.
    '''
    # 1 card played means holder is opener
    if len(trick) == 1:
        return trick[0]

    top_trump = 0
    # check if trump holds      
    for card in trick:
        if is_trump(card):
            if card > top_trump:
                top_trump=card
    if top_trump > 0:
        return top_trump

    # find holder in color -> there can be no trump in trick except Trump Fool
    else:
        color=get_color(trick[0]) if get_color(trick[0]) is not 0 else get_color(trick[1])
        # cards are valid if belonging to the color called for
        valid_cards = [card for card in trick if get_color(card) is color]
        sort_cards(valid_cards)
        return valid_cards[-1]


### TESTS ###
for i in range(3):
    trick = random.sample(PLAYING_CARDS, random.randint(2,4))
    print(f"Trick is : {trick}")
    print(f"Holding card is {to_string(holder(trick))}\n")

Trick is : [8, 'S7', 'S3']
Holding card is Trump 8

Trick is : ['S6', 3]
Holding card is Trump 3

Trick is : [18, 16]
Holding card is Trump 18



In [6]:
class Tricks():
    '''
    Class used to easily access data about past tricks.
    '''
    def __init__(self):
        self.tricks=[[]]*18
        self.winner_role=18*[0]        
        self.tricks_color=18*['']
        self.current_trick=0
        
        # ex tricks_color : { 'H' : 0, 'C' : 3, 'D' : 1, 'S' : 0}
        self.color_turns = { color:0 for color in COLOR_VALUES }

        # ex cut_colors : 'TAKER' : { 'H' : False, 'C' : False, 'D' : True, 'S' : False}
        self.cut_colors = { role: { color:False for color in COLOR_VALUES } for role in ROLES }
    
    def add_trick(self,trick,roles):
        '''
        Adds a new trick to the object -> all fields are updated with trick data
        '''
        turn=self.current_trick
        print(f"[Tricks.add_trick] turn is {turn}")
        self.tricks[turn]=trick
        self.winner_role[turn]=roles[trick.index(holder(trick))]
        print(f"[Tricks.add_trick] holder is {roles[trick.index(holder(trick))]} - taker's index is {roles.index('TAKER')}")        
        print(f"[Tricks.add_trick] trick is {trick}")        
        self.tricks_color[turn]=get_color(trick[0]) if trick[0]!=0 else get_color(trick[1])
        
        if self.tricks_color[turn] is not None:
            self.color_turns[self.tricks_color[turn]]=self.color_turns[self.tricks_color[turn]]+1

        # update cuts
        if self.tricks_color[turn] is not None:
            for i in range(4):
                if is_trump(trick[i]):
                    self.cut_colors[roles[i]][self.tricks_color[turn]]=True
                    #self.cut_colors[roles[i]][self.tricks_color[turn]]+1
        self.current_trick=turn+1
    
    def to_string(self):
        '''
        Displays the content of each field in Tricks
        '''
        print(f"\nturn : {self.current_trick}")
        print(f"tricks : {self.tricks}")
        print(f"winner : {self.winner_role}")
        print(f"color : {self.tricks_color}")
        # print(f"color turns : {self.color_turns}")
        print(f"cuts : {self.cut_colors}")
    
    def master(self, color):
        '''
        Returns the strongest card remaining in color        
        '''
        # TODO take the dog into account
        card_color_in_tricks = [card for trick in self.tricks for card in trick if get_color(card) is color]
        color_cards = [card for card in COLOR_CARDS if get_color(card) is color]

        color_cards.sort(key=get_index,reverse=True)
        for card in color_cards:
            if card not in card_color_in_tricks:
                return card
    
    def trump_master(self):
        '''
        Returns the trump master card
        '''
        trumps_in_tricks = [card for trick in self.tricks for card in trick if is_trump(card)]
        trumps=TRUMPS.copy()
        trumps.sort(reverse=True)
        
        for trump in trumps:
            if trump not in trumps_in_tricks:
                return trump
    
    def determine_trump_mastery(self,hand):
        '''
        Returns the number of trumps in hand that are master
        ex: having 21-20-19-17 at game start returns 3 (21-20-19 are untakable, 18 takes 17)
        '''
        trumps_in_tricks = [card for trick in self.tricks for card in trick if is_trump(card)]
        trumps=TRUMPS.copy()
        trumps.sort(reverse=True)
        
        masters=[]
        for trump in trumps:
            if trump not in trumps_in_tricks:
                if trump in hand:
                    masters.append(trump)
                else: 
                    return masters
    
    def compute_final_score(self, ecart):
        '''
        returns the sum of taker tricks + his dog if not empty
        '''
        # TODO Trump Fool must go back to his owner !!!
        taker_cards=[]
        if ecart is not []:
            for card in ecart:
                taker_cards.append(card)
        # print(f"[Tricks.compute_final_score] reference list is {self.winner_role}")
        for i in range(0,18):
            if self.winner_role[i] == 'TAKER':
                for card in self.tricks[i]:
                    taker_cards.append(card)
        print(f"[Tricks.compute_final_score] Taker got {self.winner_role.count('TAKER')} tricks\necart={taker_cards[:6]}\ntricks={taker_cards[6:]}")
        return compute_score(taker_cards), get_contract(taker_cards)

    def compute_cut_risk(self,color,role):
        '''
        Coputes the risk for a color to get cut
        Risk 0 is safe -> risk 100 is unsafe
        '''
        risk=0
        # TODO take hand cards and dog into account
        if role is 'TAKER':
            # TAKER won't play color's figures if getting cut
            for player in self.cut_colors:
                if self.cut_colors[player][color]:
                    risk=100
            # the more turns played the higher the risk
            if risk==0:
                risk=33*self.color_turns[color]
        else:
            # defense needs to open colors and then can moderately take risks
            if self.color_turns[color]==0:
                risk=70-25*sum(value == True for value in self.cut_colors['TAKER'].values())
            else:
                # color has been played or opened
                if self.cut_colors['TAKER'][color]:
                    risk = 100
                # less risk if allies cut
                # TODO : is that ok ?? ^^
                else:
                    allies_cutting=sum([1 for role in ROLES if self.cut_colors[role][color]] ) 
                    risk = 25*self.color_turns[color] - 25*(allies_cutting) -25*sum(value == True for value in self.cut_colors['TAKER'].values())
        return risk

### TESTS ###
trick1 = ['S6', 18, 19, 'SP']
roles1 = ['OPENER','TAKER','NEUTRAL','RELAUNCHER']
trick2 = ['C3', 'C4', 'CK', 'C9']
roles2 = ['NEUTRAL','OPENER','TAKER','RELAUNCHER']
trick3 = ['D2', 'D3', 'D1', 'DQ']
roles3 = ['TAKER','RELAUNCHER','NEUTRAL','OPENER']
trick4 = [8, 11, 13, 21]
roles4 = ['OPENER','TAKER','RELAUNCHER','NEUTRAL']

tricks = Tricks()
tricks.add_trick(trick1,roles1)
tricks.add_trick(trick2,roles2)
tricks.add_trick(trick3,roles3)
tricks.add_trick(trick4,roles4)
tricks.to_string()

print(f"\nMaster of Clubs : {tricks.master('C')}")
print(f"Master of Trumps : {tricks.trump_master()}")

hand = [3, 14, 15, 17, 20, 'H4', 'H9', 'HQ', 'D7', 'D10', 'S1']
print(f"\nHand is : {hand}")
print(f"Trump mastery of hand is {tricks.determine_trump_mastery(hand)}\n")

print(tricks.color_turns)
print(tricks.compute_cut_risk('S','TAKER'))
print(tricks.compute_cut_risk('H','TAKER'))
print(tricks.compute_cut_risk('D','TAKER'))
print(tricks.compute_cut_risk('D','NEUTRAL'))

[Tricks.add_trick] turn is 0
[Tricks.add_trick] holder is NEUTRAL - taker's index is 1
[Tricks.add_trick] trick is ['S6', 18, 19, 'SP']
[Tricks.add_trick] turn is 1
[Tricks.add_trick] holder is TAKER - taker's index is 2
[Tricks.add_trick] trick is ['C3', 'C4', 'CK', 'C9']
[Tricks.add_trick] turn is 2
[Tricks.add_trick] holder is OPENER - taker's index is 0
[Tricks.add_trick] trick is ['D2', 'D3', 'D1', 'DQ']
[Tricks.add_trick] turn is 3
[Tricks.add_trick] holder is NEUTRAL - taker's index is 1
[Tricks.add_trick] trick is [8, 11, 13, 21]

turn : 4
tricks : [['S6', 18, 19, 'SP'], ['C3', 'C4', 'CK', 'C9'], ['D2', 'D3', 'D1', 'DQ'], [8, 11, 13, 21], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
winner : ['NEUTRAL', 'TAKER', 'OPENER', 'NEUTRAL', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
color : ['S', 'C', 'D', None, '', '', '', '', '', '', '', '', '', '', '', '', '', '']
cuts : {'TAKER': {'H': False, 'C': False, 'D': False, 'S': True}, 'OPENER': {'H': False, 'C': False, 'D': Fals

## Biding

In [7]:
CONTRACTS = [56,51,41,36]
POINTS = [1.5,2.5,3.5,4.5]

# CARD + SCORING FUNCTION
def get_card_points(card):
    if is_trump(card):        
        return 4.5 if card in OUDLERS else .5
    else:
        return .5 if card[1] in NON_FIGURES else POINTS[FIGURES.index(card[1])]

# HAND + SCORING FUNCTION
def compute_score(cards):
    return sum(get_card_points(card) for card in cards)

# HAND + SCORING FUNCTION
def get_contract(hand):
    return CONTRACTS[sum([1 for card in hand if is_oudler(card)])]

# # HAND + SCORING FUNCTION
# def estimate_win_factor(hand):
#     '''
#     2 essential params : number of trump cards + actual score
#     each has .5 potential influence -> the closest to 1, the higher the chances of victory
#     '''
#     # compare actual points vs needed
#     # TODO : check if hands valued under 1 are worth taking
#     score_param = .5 + .02*(compute_score(hand) + 10 - get_contract(hand))
    
#     # ratio of actual trump count VS mean    
#     trump_mean = 0 if get_trump_count(hand)==0 else sum([10 if card is 0 else card for card in hand if is_trump(card)]) / get_trump_count(hand)

#     # TODO : biggest trumps have more value
#     trump_param = .5 + 0.05*(get_trump_count(hand)-22/4)
    
#     return score_param + trump_param

## Playing

There are 3 main roles in tarot : Taker - Opener - Relauncher

Taker gameplay :
- play 'longes' -> play many cards of the same color so trumps fall, then taker's trumps are master
- play figures 'safely'
- make trumps fall -> play medium trumps so bigger may fall, and taker's trumps will be master later on

==== DEFENSE ROLES ====

Opener gameplay :
- open and re-open colors (be the first player playing the color) where figures are left IF taker doesn't cut

Relauncher gameplay :
- relaunch already opened colors (play only the color that have been played once)

Neutral can be introduced too :
- gameplay is between opener and relauncher -> he should open if needed, but prefers relaunching

In [15]:
class Player():
    def __init__(self, hand, *args, **kwargs):
        '''
        A player is defined by :
        - his role (as taker or in the defense)
        - his hand (initially 18 cards)
        '''
        self.role = kwargs.get('role',None)
        self.hand = hand.copy()

    def play_card(self,current_trick,taker_id,tricks,**kwargs):
        '''
        Returns the choice of a card from the player.
        Removes the chosen card from his hand.
        '''
        is_first_player=kwargs.get("is_first",False)
        if is_first_player:
            card = self.open(tricks)
            if card is None:
                print(f"[Player.play_card] ERROR IN OPEN FUNC -> returned NONE")
        else:
            card = self.answer(current_trick,taker_id,tricks)
            if card is None:                
                print(f"[Player.play_card] ERROR IN ANSWER FUNC -> returned NONE")
        # print(f"[Player.play_card] {self.role} hand is : {self.hand}")
        # print(f"[Player.play_card] chosen card is : {card}")
        self.hand.remove(card)
        return card

    # TRICK + PLAYER FUNCTION
    def open(self, tricks):
        '''
        Implements the way of opening for each 3 role
        '''
        # masters = [master(color,tricks) for color in COLOR_VALUES]
        masters = [tricks.master(color) for color in COLOR_VALUES]
        # print(f"[Player.open] masters are : {masters}")

        if self.role is 'TAKER':
            # try to play strong figures
            # TODO modify ! this is stupid
            for card in self.hand:
                if card in masters:
                    if card is None:
                        print('[Player.open] ERROR 1')
                    print(f"[Player.open] TAKER opens with STRONG FIGURE")
                    return card

            # try to play 'longe'
            for color in COLOR_VALUES:
                color_cards_in_hand = get_color_cards(self.hand,color)
                color_mean = 14-sum([1 for trick in tricks.tricks for card in trick if get_color(card) is color])-len(color_cards_in_hand)/3
                if len(color_cards_in_hand) > color_mean + 3:
                    if get_card_points(card) < 2.5:
                        card=color_cards_in_hand[0]         
                        if card is None:
                            print('[Player.open] ERROR 2')
                        print(f"[Player.open] TAKER opens with PLAY LONGE")
                        return card

            # make trumps fall
            #*3/7 makes player play his 3rd trump when having 7, which should be acceptable
            # TODO check if trump value is coherent (7<val<12 is optimal)
            trump_count = get_trump_count(self.hand)
            trump_mean = (22 - sum([1 for trick in tricks.tricks for card in trick if is_trump(card)]) - trump_count)/3
            if trump_count >  trump_mean + 3:                
                card=get_trump_cards(self.hand)[int(trump_count*3/7)]
                if card is None:
                    print('[Player.open] ERROR 3')
                print(f"[Player.open] TAKER decides to force trump fall")
                return card

            # TODO define what else can be done
            print(f'[Player.open] {self.role} opened with random card')
            card = random.choice(self.hand)
            if card is None:
                print('[Player.open] ERROR 6')
            return card

        elif self.role is 'OPENER':            
            tricks.color_turns
            # try to open new color
            for color in tricks.color_turns:
                if tricks.color_turns[color] == 0:
                    color_cards_in_hand = get_color_cards(self.hand,color)
                    good_opener = [color_card for color_card in color_cards_in_hand if not is_figure(color_card)]
                    if len(good_opener) > 0:
                        card=good_opener[-1]
                        if card is None:
                            print('[Player.open] ERROR 4')
                        print(f"[Player.open] {self.role} opens NEW COLOR {get_color(card)}")
                        return card
            # TODO define what else can be done
            card = random.choice(self.hand)
            if card is None:
                print('[Player.open] ERROR 6')
            print(f"[Player.open] {self.role} opens with RANDOM CARD")
            return card
        else:
            # roles other than TAKER and OPENER
            # TODO change this !!!
            card = random.choice(self.hand)
            if card is None:
                print('[Player.open] ERROR 5')            
            print(f"[Player.open] {self.role} opens with RANDOM CARD")
            return card
            
        #     # re-open colors
        #     # TODO manage taker cutting
        #     # TODO manage usefulness of opening (figures left)
        #     for color in hand:
        #         if figures fall count < 3:
        #             return re-open color

        #     # TODO define what else can be done
        #     return random.choice(hand)

        # elif self.role is 'RELAUNCHER':
        #     # try to relaunch color
        #     # try to get information if figures in color

    # TRICK + PLAYER FUNCTION
    def answer(self, current_trick, taker_id, tricks):
        '''
        If a player doesn't open, he has to answer to the opening card
        '''
        opening = current_trick[0]

        # Answer to a trump opening
        if is_trump(opening):
            if get_trump_count(self.hand) > 0:
                print(f"[Player.answer] {self.role} has to overtrump with one of {get_trump_count(self.hand)} trumps")
                return self.cutOrClimb(current_trick,taker_id) if opening!=0 else self.open(tricks)
            else:
                print(f"[Player.answer] player can't play trump and discards")
                return self.discard(current_trick,taker_id,tricks)
        # Answer to a color opening
        else:            
            color=get_color(opening)
            cards_in_color=get_color_cards(self.hand,color)            
            # print(f"[Player.answer] opening color is {color} and cards_in_color is {cards_in_color}")
            if len(cards_in_color)>0:
                # TODO compute risk of playing figure
                # print(f"[Player.answer] {self.role} CAN answer in color")
                if len(cards_in_color)==1:
                    print(f"[Player.answer] {self.role} CAN answer in color - plays LAST CARD in color")
                    return cards_in_color[0] 
                else: 
                    return self.play_color(current_trick,cards_in_color,color,tricks,taker_id)

            # cut : player has trumps, he must use them
            elif get_trump_count(self.hand) > 0:
                print(f"[Player.answer] {self.role} CAN NOT answer : he trumps or overtrumps")
                return self.cutOrClimb(current_trick,taker_id)

            # discard : player has no trump, nor cards in color
            else:
                print(f"[Player.answer] {self.role} CAN NOT answer nor trump : he discards")
                return self.discard(current_trick,taker_id,tricks)

    # TRICK + PLAYER FUNCTION
    def play_color(self,current_trick,cards_in_color,color,tricks,taker_id):
        '''
        Player has to answer in color if he can except if he has Trump Fool.
        He might want to take risk, or not.        
        '''
        holding_card = holder(current_trick)
        # taker is safe if last
        if self.role is 'TAKER':
            safe = len(current_trick) is 3
        # player isnt taker -> safe if taker hasnt played
        # TODO take cuts into account (what if RELAUNCHER cuts ?!)
        else:
            safe = len(current_trick)>taker_id
        
        # risk 0 is safe -> risk 100 is unsafe
        risk = tricks.compute_cut_risk(color,self.role)

        #play figure if safe and figure can master trick
        #TODO what if ally holds trick ?!
        if safe and get_card_value(holding_card) < get_card_value(cards_in_color[-1]) and is_figure(cards_in_color[-1]):  
            print(f"[Player.answer] {self.role} CAN answer in color - plays FIGURE")
            return cards_in_color[-1]
        # play figure if not too risky and master in color
        elif tricks.master(color)==cards_in_color[-1] and risk < 50 and is_figure(cards_in_color[-1]):
            return cards_in_color[-1]
        else:
            print(f"[Player.answer] {self.role} CAN answer in color - plays LOWEST card in color")
            return cards_in_color[0]
            
        print(f"[Player.answer] {self.role} CAN answer in color with RANDOM")
        return random.choice(cards_in_color)
        
    
    # TRICK + PLAYER FUNCTION
    def cutOrClimb(self,current_trick,taker_id):
        '''Player has to trump or overtrump.'''    
        player_trumps = get_trump_cards(self.hand)

        # player has 1 trump : he must play it
        # TODO take fool into consideration
        if len(player_trumps) is 1:
            return player_trumps[0]

        trick_trumps = [card for card in current_trick if is_trump(card)]
        ref = max(trick_trumps) if len(trick_trumps) > 0 else 0
        
        # TODO define what is safe to TAKER
        # safe = False if taker_id > len(current_trick) else not holder(current_trick)==current_trick[taker_id]
        safe = False if taker_id > len(current_trick) else True if taker_id==len(current_trick) else not holder(current_trick)==current_trick[taker_id]

        # climb or cut
        if max(player_trumps) > ref:
            # try play 1
            if 1 in player_trumps and safe and 1>ref:
                return 1
            for trump in player_trumps:
                if trump > ref: return trump
        # discard
        else:
            #try play 1
            if 1 in player_trumps and safe:
                return 1
            #discard anything
            else:
                sort_cards(player_trumps)
                for card in player_trumps:
                    if card != 1:
                        return card
                return player_trumps[0]

    # TRICK + PLAYER FUNCTION
    def discard(self,current_trick,taker_id,tricks):
        '''
        Never safe to discard as TAKER.
        Safe to discard if TAKER not holding.
        If SAFE -> Policy is to discard FIGURES with priority given to cut colors.
        '''
        # TODO update cut_color for player (as long as he has no more trumps !)                    
        if sum(value == True for value in tricks.cut_colors[self.role].values())==0:
            for color in COLOR_VALUES:                
                tricks.cut_colors[self.role][color]=False
            # print(f"[Player.discard] {self.role} discarded and got his cuts reset")

        safe = False if taker_id >= len(current_trick) else not holder(current_trick)==current_trick[taker_id]
        
        if safe:
            # TODO is discarding figures the best option ? what if taker out of trumps ?
            # discard figure in color cut by TAKER
            for color in tricks.cut_colors['TAKER']:
                if tricks.cut_colors['TAKER'][color]:
                    color_cards=get_color_cards(self.hand,color)
                    # check if player has cards in cut colors
                    if len(color_cards)>0:
                        color_cards.sort(key=get_index,reverse=True)
                        # play figures
                        for card in color_cards:
                            if is_figure(card):
                                print(f'[discard] {self.role} chose to discard in cut color')
                                return card
            #TODO get an idea of the game state : losing ? winning ? if losing then discard other figures !
            # for now we simply discard any possible figure
            for color in COLOR_VALUES:
                color_cards=get_color_cards(self.hand,color)
                # check if player has cards in cut colors
                if len(color_cards)>0:
                    color_cards.sort(key=get_index,reverse=True)
                    # play figures
                    for card in color_cards:
                        if is_figure(card):
                            print(f'[discard] {self.role} choses to discard figure')
                            return card
            # TODO default will be discarding lowest -> get more value
        else:
            # TODO think about what to do if unsafe ?
            # ex : discard card in color cut by TAKER
            pass
        
        # player has nothing of value : he discards his lowest card
        lowest=self.hand[0]
        for card in self.hand:
            if get_card_value(card)<get_card_value(lowest):
                lowest = card
        print(f'[Player.discard] {self.role} chose to discard his lowest card')
        return lowest

In [16]:
### TESTS over PLAYER ###
for i in range(5):
    sampling = random.sample(PLAYING_CARDS, random.randint(12,14))
    hand=sampling[:10]
    sort_cards(hand)
    myPlayer=Player(hand)
    trick=sampling[10:]
    if get_trump_count(hand)>0:
        print(f"\nHand : {hand}")
        print(f"Trick : {trick}")
        print(f"Choice: {myPlayer.cutOrClimb(trick,1)}")

trick1 = ['S6', 18, 19, 'SP']
roles1 = ['OPENER','TAKER','NEUTRAL','RELAUNCHER']
trick2 = ['C3', 'C4', 'CK', 'C9']
roles2 = ['NEUTRAL','OPENER','TAKER','RELAUNCHER']
trick3 = ['D2', 'D3', 'D1', 'DQ']
roles3 = ['TAKER','RELAUNCHER','NEUTRAL','OPENER']
trick4 = [8, 11, 13, 21]
roles4 = ['OPENER','TAKER','RELAUNCHER','NEUTRAL']

tricks = Tricks()
tricks.add_trick(trick1,roles1)
tricks.add_trick(trick2,roles2)
tricks.add_trick(trick3,roles3)
tricks.add_trick(trick4,roles4)
tricks.to_string()

# print()
# for player in tricks.cut_colors:
#     print(f"Player cut colors : {tricks.cut_colors[player][color]}")
# print()

print()
hand = ['H4', 'H9', 'HQ', 'D7', 'D10', 'S1']
myPlayer = Player(hand,role='NEUTRAL')
print(hand)
print(myPlayer.discard(['S8', 2,'SP'],1,tricks))

hand = ['H4', 'H9', 'D7', 'D10', 'S1']
myPlayer = Player(hand,role='NEUTRAL')
print(hand)
print(myPlayer.discard(['S8', 2,'SP'],1,tricks))

hand = ['H4', 'H9', 'D7', 'D10', 'S1']
myPlayer = Player(hand,role='NEUTRAL')
print(hand)
print(myPlayer.answer(['S8', 2,'SP'],1,tricks))


Hand : [2, 16, 17, 20, 'H5', 'C4', 'C6', 'C8', 'D9', 'S6']
Trick : ['H1', 'D6', 'DQ']
Choice: 2

Hand : [1, 'H6', 'HJ', 'HP', 'HK', 'C6', 'C8', 'S3', 'SQ', 'SK']
Trick : ['D3', 'S4']
Choice: 1

Hand : [3, 7, 10, 'H1', 'HP', 'D8', 'S1', 'S8', 'S9', 'SK']
Trick : ['D1', 'C7', 'HJ', 19]
Choice: 3

Hand : [0, 3, 6, 'H3', 'H9', 'HJ', 'DP', 'DK', 'S6', 'S8']
Trick : [5, 'S4', 'D3']
Choice: 6

Hand : [1, 9, 18, 'H10', 'C8', 'C10', 'D7', 'D9', 'DP', 'S5']
Trick : ['H4', 'D4', 'S8']
Choice: 1
[Tricks.add_trick] turn is 0
[Tricks.add_trick] holder is NEUTRAL - taker's index is 1
[Tricks.add_trick] trick is ['S6', 18, 19, 'SP']
[Tricks.add_trick] turn is 1
[Tricks.add_trick] holder is TAKER - taker's index is 2
[Tricks.add_trick] trick is ['C3', 'C4', 'CK', 'C9']
[Tricks.add_trick] turn is 2
[Tricks.add_trick] holder is OPENER - taker's index is 0
[Tricks.add_trick] trick is ['D2', 'D3', 'D1', 'DQ']
[Tricks.add_trick] turn is 3
[Tricks.add_trick] holder is NEUTRAL - taker's index is 1
[Tricks.ad

In [25]:
class Game():
    """DOC"""

    def __init__(self, *args, **kwargs):
        """DOC"""
        hand0,hand1,hand2,hand3,dog=self.deal()
        self.hands = [hand0,hand1,hand2,hand3]
        for hand in self.hands:
            sort_cards(hand)
        self.dog = dog
        self.ecart=6*[0]
        self.players = [Player(hand) for hand in self.hands]
        
        self.contract=None
        self.dealer = 0
        self.first_player = 1
        self.tricks=Tricks()
        self.taker_id=-1

    # GAME FUNCTION
    def deal(self):
        '''Returns 4 hands of 18 cards + a dog of 6 cards'''
        deck = PLAYING_CARDS.copy()
        random.shuffle(deck)
        return [deck[0:18],deck[18:36],deck[36:54],deck[54:72],deck[72:78]]

    # GAME FUNCTION
    def bid(self):
        '''
        Computes win_factor for each player
        Returns taker_id, announces
        '''
        current_bid=BID_VALUES[0]
        player_bids = [0,0,0,0]
        for i in range(4):
            win_factor = self.estimate_win_factor(self.players[i].hand)
            if win_factor<.9:
                pass
            else:
                # TAKE
                if win_factor < QUANTILES[3] and current_bid<BID_VALUES[1]:
                    player_bids[i]=BID_VALUES[1]
                    current_bid=BID_VALUES[1]
                    self.contract='TAKE'
                    self.taker_id=i
                # PUSH
                elif win_factor < QUANTILES[5] and current_bid<BID_VALUES[2]:
                    player_bids[i]=BID_VALUES[2]
                    current_bid=BID_VALUES[2]
                    self.contract='PUSH'
                    self.taker_id=i
                # GUARD
                elif win_factor < QUANTILES[7] and current_bid<BID_VALUES[3]:
                    player_bids[i]=BID_VALUES[3]
                    current_bid=BID_VALUES[3]
                    self.contract='GUARD'
                    self.taker_id=i
                # GUARD WITHOUT
                elif win_factor < QUANTILES[8] and current_bid<BID_VALUES[4]:
                    player_bids[i]=BID_VALUES[4]
                    current_bid=BID_VALUES[4]
                    self.contract='GUARD WITHOUT'
                    self.taker_id=i
                # GUARD AGAINST
                elif current_bid<BID_VALUES[5]:
                    player_bids[i]=BID_VALUES[5]
                    current_bid=BID_VALUES[5]
                    self.contract='GUARD AGAINST'
                    self.taker_id=i
                    
        if max(player_bids) is 0: 
            print("[Game.bid] No one took.")
        else:
            for i in range(4):
                self.players[(player_bids.index(max(player_bids))-i)%4].role=ROLES[i%4]
        
        return player_bids.index(max(player_bids)),[BIDS[BID_VALUES.index(bid)] for bid in player_bids]

    # HAND + SCORING FUNCTION
    def estimate_win_factor(self, hand):
        '''
        2 essential params : number of trump cards + actual score
        each has .5 potential influence -> the closest to 1, the higher the chances of victory
        '''
        # compare actual points vs needed
        # TODO : check if hands valued under 1 are worth taking
        score_param = .5 + .02*(compute_score(hand) + 10 - get_contract(hand))
        
        # ratio of actual trump count VS mean    
        trump_mean = 0 if get_trump_count(hand)==0 else sum([10 if card is 0 else card for card in hand if is_trump(card)]) / get_trump_count(hand)

        # TODO : biggest trumps have more value
        trump_param = .5 + 0.05*(get_trump_count(hand)-22/4)
        
        return score_param + trump_param

    def incorporate_dog(self,player):
        '''
        Add 6 dog cards to TAKER hand.
        Then removes 6 cards from TAKER hand.
        '''
        #TODO incorporate DEPENDS on CONTRACT

        if self.contract is 'GUARD AGAINST':
            self.ecart=[]
            return 0
        elif self.contract is 'GUARD WITHOUT':
            self.ecart=self.dog
            return 0    

        for card in self.dog:
            player.hand.append(card)

        ecartables=[self.is_ecartable(get_color_cards(player.hand,color)) for color in COLOR_VALUES]

        print(f"[Game.incorporate_dog] Color {COLOR_VALUES[ecartables.index(max(ecartables))]} chosen for ecart of type [CUT or KING CINGLETTE]: {get_color_cards(player.hand,COLOR_VALUES[ecartables.index(max(ecartables))])}")
        # TODO what else can be done ?
        ecart_count=0
        chosen_color=COLOR_VALUES[ecartables.index(max(ecartables))]

        # ecart first color
        for card in get_color_cards(player.hand,COLOR_VALUES[ecartables.index(max(ecartables))]):
            if not get_card_role(card)=='K':
                self.ecart[ecart_count]=card
                player.hand.remove(card)
                ecart_count=ecart_count+1
        
        new_ecartables=[self.is_ecartable(get_color_cards(player.hand,color)) for color in COLOR_VALUES]

        # ecart second color
        ecart_2 = False
        if ecart_count < 6:
            for color in COLOR_VALUES:
                if color is not chosen_color:
                    if new_ecartables[COLOR_VALUES.index(color)]>.6 and len(get_color_cards(player.hand,color))<=6-ecart_count:
                        print(f"[Game.incorporate_dog] Second color {color} was chosen for ecart [CUT or KING CINGLETTE] : {get_color_cards(player.hand,color)}")
                        ecart_2 = True
                        for card in get_color_cards(player.hand,color):
                            if not get_card_role(card)=='K':
                                self.ecart[ecart_count]=card
                                player.hand.remove(card)
                                ecart_count=ecart_count+1
        
        # create cinglette
        if ecart_2 == False and ecart_count<6:
            for color in COLOR_VALUES:
                if color is not chosen_color:
                    if new_ecartables[COLOR_VALUES.index(color)]>.6 and len(get_color_cards(player.hand,color))<=7-ecart_count:
                        print(f"[Game.incorporate_dog] Second color {color} was chosen for ecart [CINGLETTE] : {get_color_cards(player.hand,color)}")
                        cards=get_color_cards(player.hand,color)
                        cards.sort(key=get_index,reverse=True)
                        for card in cards:
                            if not get_card_role(card)=='K' and is_figure(card):
                                self.ecart[ecart_count]=card
                                player.hand.remove(card)
                                ecart_count=ecart_count+1
                        cards.sort(key=get_index,reverse=False)
                        for card in cards:
                            if ecart_count<6:
                                self.ecart[ecart_count]=card
                                player.hand.remove(card)
                                ecart_count=ecart_count+1

        # TODO do a better ecart by removing orphan figures (such as knight or jack)
        while ecart_count<6:
            card=random.choice(player.hand)
            # pick random cards except kings and trumps
            if not is_trump(card) and not get_card_role(card)=='K':
                self.ecart[ecart_count]=card
                print(f"[Game.incorporate_dog] card {card} was picked RANDOMLY to complete the ecart")
                player.hand.remove(card)
                ecart_count=ecart_count+1

    def is_ecartable(self, cards):
        '''
        Computes if a color can be fully put in ecart.
        Empirical threshold is 0.6 : 1 means color is ecartable.
        '''
        # ecart if count_factor close to 1
        count_factor=(14-len(cards))/14 if len(cards) < 7 else -.5
        # ecart if score_factor close to 1
        score_factor=((4.5+3.5+2.5+2.5)-compute_score([card for card in cards if is_figure(card)]))/(4.5+3.5+2.5+2.5)
        # score has more value than nb of cards
        return (count_factor+2*score_factor)/3

    # GAME FUNCTION
    def play_trick(self):
        '''
        Each player plays one card.
        First player is opener, the rest simply answer.
        The strongest determines the trick's winner.
        '''
        current_trick=[]
        roles=[0,0,0,0]
        if len(self.players[0].hand) == 18:
            self.first_player = (self.dealer+1)%4
        else:
            for player in self.players:
                if player.role is self.tricks.winner_role[18-(len(player.hand)+1)]:
                    self.first_player = self.players.index(player)
                    print(f'[Game.play_trick] Next first player should be {self.players[self.first_player].role}')

        print("\n[Game.play_trick] === NEW TRICK ===")
        for i in range(4):
            # players should play in the right order
            player_id=(self.first_player+i)%4
            player=self.players[player_id]
            taker_turn=(self.taker_id+4-self.first_player)%4+1
            # print("[Game.play_trick] displaying roles")
            # print(f"[Game.play_trick] player_id is {player_id}")
            # print(f"[Game.play_trick] current player is {player.role}")
            roles[i]=player.role
            # print(f"[Game.play_trick] taker will play as {taker_turn}")
            chosen_card=self.players[(self.first_player+i)%4].play_card(current_trick,taker_turn-1,self.tricks,is_first=((self.first_player+i)%4==self.first_player))
            # print(f"[Game.play_trick] chosen card : {chosen_card}")
            current_trick.append(chosen_card)
        # print(f"[Game.play_trick] called function [tricks.add_trick] over trick={current_trick} and roles {roles}")
        self.tricks.add_trick(current_trick,roles)
        # print(f"[Game.play_trick] winner_role list is now : {self.tricks.winner_role}")

### TESTS ###
myGame = Game()
bids = myGame.bid()
print(f"[Game.TEST] bids are {bids[1]}")

if bids[1].count('PASS')!=4:
    for player in myGame.players:
        if player.role is 'TAKER':
            myGame.taker_id=myGame.players.index(player)
            print(f"[Game.TEST] TAKER estimated win factor is : {myGame.estimate_win_factor(player.hand)} (max expected is around 1.25, below 1 is more risky)")
            # print(f"[Game.__init__]found {player.role}, id is {self.taker_id}")
    print(f"[Game.TEST] displaying roles {[player.role for player in myGame.players]}")

    myGame.incorporate_dog(myGame.players[myGame.taker_id])
    sort_cards(myGame.ecart)
    if myGame.contract not in ['GUARD WITHOUT','GUARD AGAINST']:
        print(f"[Game.TEST] TAKER has elected ecart={myGame.ecart}")
    myTakerHand=myGame.players[myGame.taker_id].hand
    sort_cards(myTakerHand)
    print(f"[Game.TEST] TAKER hand is {myTakerHand}")

    for i in range(18):
        myGame.play_trick()
    # myGame.tricks.to_string()
    score,contract=myGame.tricks.compute_final_score(myGame.ecart)
    print(f'\nTaker final score is {score} with contract {contract}')
    if score>=contract:
        print('Taker wins !')
    else:
        print('Defense wins !')
# print(player.role for player in myGame.players)

[Game.TEST] bids are ['TAKE', 'PASS', 'PASS', 'PASS']
[Game.TEST] TAKER estimated win factor is : 0.9650000000000001 (max expected is around 1.25, below 1 is more risky)
[Game.TEST] displaying roles ['TAKER', 'RELAUNCHER', 'NEUTRAL', 'OPENER']
[Game.incorporate_dog] Color C chosen for ecart of type [CUT or KING CINGLETTE]: ['C5', 'C6']
[Game.incorporate_dog] Second color S was chosen for ecart [CUT or KING CINGLETTE] : ['S6', 'S7']
[Game.incorporate_dog] card H10 was picked RANDOMLY to complete the ecart
[Game.incorporate_dog] card D5 was picked RANDOMLY to complete the ecart
[Game.TEST] TAKER has elected ecart=['H10', 'C5', 'C6', 'D5', 'S6', 'S7']
[Game.TEST] TAKER hand is [0, 1, 5, 10, 11, 14, 15, 18, 'H1', 'H3', 'H4', 'H5', 'H6', 'HP', 'D1', 'DP', 'DQ', 'DK']

[Game.play_trick] === NEW TRICK ===
[Player.open] RELAUNCHER opens with RANDOM CARD
[Player.answer] NEUTRAL CAN answer in color - plays LOWEST card in color
[Player.answer] OPENER CAN answer in color - plays LOWEST card in col